In [100]:
import requests
import json
import os
import pandas as pd

from urllib.parse import quote_plus, unquote_plus
from requests.utils import requote_uri
from warcio import ArchiveIterator


In [23]:
def search_ru_wiki(index_name):
	url = 'ru.wikipedia.org/*'
	encoded_url = quote_plus(url)
	index_url = f'http://index.commoncrawl.org/{index_name}-index?url={encoded_url}&output=json'
	response = requests.get(index_url)

	if response.status_code == 200:
		records = response.text.strip().split('\n')
		return [json.loads(record) for record in records]
	else:
		return None


In [66]:
results = search_ru_wiki('CC-MAIN-2024-38')
results += search_ru_wiki('CC-MAIN-2024-33')
results += search_ru_wiki('CC-MAIN-2024-30')
results += search_ru_wiki('CC-MAIN-2024-26')
results += search_ru_wiki('CC-MAIN-2024-22')

In [67]:
len(results)

66298

In [71]:
unique_results = {}
for result in results:
	if result['url'] not in unique_results:
		unique_results[result['url']] = result

len(unique_results)

65102

In [54]:
def fetch_single_record(warc_record_filename, offset, length):

    s3_url = f'https://data.commoncrawl.org/{warc_record_filename}'

    byte_range = f'bytes={offset}-{offset + length - 1}'

    response = requests.get(
        s3_url,
        headers={'Range': byte_range},
        stream=True
    )

    if response.status_code == 206:

        stream = ArchiveIterator(response.raw)
        for warc_record in stream:
            if warc_record.rec_type == 'response':
                return warc_record.content_stream().read()
    else:
        print(f"Failed to fetch data: {response.status_code}")
    
    return None

In [105]:
keywords = ['Перм', 'Пастернак', 'ПНИПУ', 'МФТИ', 'МГУ', 'Московский государственный университет', 'Московский физико-технический институт', 'Московский институт физики и технологии', 'Московский государственный университет имени М.В. Ломоносова', 'Московский государственный университет имени Михаила Васильевича Ломоносова', 'МГУ имени М.В. Ломоносова', 'МГУ имени Михаила Васильевича Ломоносова'] 

keywords

['Перм',
 'Пастернак',
 'ПНИПУ',
 'МФТИ',
 'МГУ',
 'Московский государственный университет',
 'Московский физико-технический институт',
 'Московский институт физики и технологии',
 'Московский государственный университет имени М.В. Ломоносова',
 'Московский государственный университет имени Михаила Васильевича Ломоносова',
 'МГУ имени М.В. Ломоносова',
 'МГУ имени Михаила Васильевича Ломоносова']

In [102]:
filtered_results = []
for result in unique_results.values():
	if any(requote_uri(keyword.casefold()) in result['url'].casefold() for keyword in keywords):
		filtered_results.append(result)

len(filtered_results)


14

In [96]:
html_results = {}

In [97]:
for result in filtered_results:
	record = fetch_single_record(result['filename'], int(result['offset']), int(result['length']))
	if record:
		html_results[result['url']] = record


In [109]:
from bs4 import BeautifulSoup


for url, html in html_results.items():
	beautiful_soup = BeautifulSoup(html, 'html.parser')
	print(f"Статья - {beautiful_soup.title.string}; URL - {unquote_plus(url)}")

	for keyword in keywords:
		if keyword.casefold() in beautiful_soup.get_text().casefold():
			print(f"Ключевое слово: {keyword} в статье {beautiful_soup.title.string}")

	print("\n\n")



Статья - Административно-территориальное деление Пермского края — Википедия; URL - https://ru.wikipedia.org/wiki/Административно-территориальное_деление_Пермского_края
Ключевое слово: Перм в статье Административно-территориальное деление Пермского края — Википедия



Статья - Берёзовский район (Пермский край) — Википедия; URL - https://ru.wikipedia.org/wiki/Берёзовский_район_(Пермский_край)
Ключевое слово: Перм в статье Берёзовский район (Пермский край) — Википедия



Статья - Заслуженный профессор МГУ — Википедия; URL - https://ru.wikipedia.org/wiki/Заслуженный_профессор_МГУ
Ключевое слово: МГУ в статье Заслуженный профессор МГУ — Википедия
Ключевое слово: Московский государственный университет в статье Заслуженный профессор МГУ — Википедия



Статья - Ильинский район (Пермский край) — Википедия; URL - https://ru.wikipedia.org/wiki/Ильинский_район_(Пермский_край)
Ключевое слово: Перм в статье Ильинский район (Пермский край) — Википедия



Статья - Историко-филологический факультет Пер